In [ ]:
WITH 
-- 1. 年度维度：生成当前年1-12月，筛选【当前月+未来3个月】，年度核心维度
year_month AS (
    SELECT 
        YEAR(CURRENT_DATE()) AS `year`, 
        month AS `month`
    FROM (
        SELECT 1 AS month UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL 
        SELECT 7 AS month UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10 UNION ALL SELECT 11 UNION ALL SELECT 12
    ) t
    WHERE month <= MONTH(CURRENT_DATE()) + 3
),
-- 2. 未来预测日期维度【季度核心逻辑】：当前日期 至 本季度末，带15天内外标记，年度复用此范围
future_dates AS (
    SELECT 
        DATE_ADD(CURRENT_DATE(), INTERVAL n DAY) AS target_day,
        IF(n <= 14, 1, 0) AS is_15d_inner,
        YEAR(DATE_ADD(CURRENT_DATE(), INTERVAL n DAY)) AS forecast_year,
        MONTH(DATE_ADD(CURRENT_DATE(), INTERVAL n DAY)) AS forecast_month
    FROM (
        SELECT 0 n UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL 
        SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL 
        SELECT 10 UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL 
        SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19 UNION ALL 
        SELECT 20 UNION ALL SELECT 21 UNION ALL SELECT 22 UNION ALL SELECT 23 UNION ALL SELECT 24 UNION ALL 
        SELECT 25 UNION ALL SELECT 26 UNION ALL SELECT 27 UNION ALL SELECT 28 UNION ALL SELECT 29 UNION ALL 
        SELECT 30 UNION ALL SELECT 31 UNION ALL SELECT 32 UNION ALL SELECT 33 UNION ALL SELECT 34 UNION ALL 
        SELECT 35 UNION ALL SELECT 36 UNION ALL SELECT 37 UNION ALL SELECT 38 UNION ALL SELECT 39 UNION ALL 
        SELECT 40 UNION ALL SELECT 41 UNION ALL SELECT 42 UNION ALL SELECT 43 UNION ALL SELECT 44 UNION ALL 
        SELECT 45 UNION ALL SELECT 46 UNION ALL SELECT 47 UNION ALL SELECT 48 UNION ALL SELECT 49 UNION ALL 
        SELECT 50 UNION ALL SELECT 51 UNION ALL SELECT 52 UNION ALL SELECT 53 UNION ALL SELECT 54 UNION ALL 
        SELECT 55 UNION ALL SELECT 56 UNION ALL SELECT 57 UNION ALL SELECT 58 UNION ALL SELECT 59 UNION ALL 
        SELECT 60 UNION ALL SELECT 61 UNION ALL SELECT 62 UNION ALL SELECT 63 UNION ALL SELECT 64 UNION ALL 
        SELECT 65 UNION ALL SELECT 66 UNION ALL SELECT 67 UNION ALL SELECT 68 UNION ALL SELECT 69 UNION ALL 
        SELECT 70 UNION ALL SELECT 71 UNION ALL SELECT 72 UNION ALL SELECT 73 UNION ALL SELECT 74 UNION ALL 
        SELECT 75 UNION ALL SELECT 76 UNION ALL SELECT 77 UNION ALL SELECT 78 UNION ALL SELECT 79 UNION ALL 
        SELECT 80 UNION ALL SELECT 81 UNION ALL SELECT 82 UNION ALL SELECT 83 UNION ALL SELECT 84 UNION ALL 
        SELECT 85 UNION ALL SELECT 86 UNION ALL SELECT 87 UNION ALL SELECT 88 UNION ALL SELECT 89 UNION ALL 
        SELECT 90 UNION ALL SELECT 91 UNION ALL SELECT 92
    ) t
    WHERE DATE_ADD(CURRENT_DATE(), INTERVAL n DAY) >= CURRENT_DATE()
      AND DATE_ADD(CURRENT_DATE(), INTERVAL n DAY) <= LAST_DAY(STR_TO_DATE(CONCAT(YEAR(CURRENT_DATE()), '-', FLOOR((MONTH(CURRENT_DATE())-1)/3)*3 +3, '-01'),'%Y-%m-%d'))
),
-- 3. 场站基础信息【更新为新版表名】
station_info AS (
    SELECT DISTINCT
        station_code, 
        station_name, 
        station_type,
        src_region_name, 
        src_province_name
    FROM ads.ads_stnops_energy_monthly_capacity_1m_v
    WHERE year_num = YEAR(CURRENT_DATE()) AND station_code IS NOT NULL
),
-- ================ 【核心修改1 重中之重】有效场站 全年聚合版 ================
-- 去掉月份维度，按场站编码分组，取【全年最大存量/增量/累计容量】，每个场站只返回1行，彻底去重
valid_stations AS (
    SELECT
        YEAR(CURRENT_DATE()) AS capacity_year,
        station_code,
        MAX(COALESCE(stock_grid_capacity_mw, 0)) AS stock_capacity,  -- 存量取全年最大值
        MAX(COALESCE(incr_grid_capacity_mw, 0)) AS incr_capacity,  -- 增量取全年最大值
        MAX(cumulative_grid_capacity_mw) AS cumulative_grid_capacity_mw  -- 累计容量取全年最大值
    FROM ads.ads_stnops_energy_monthly_capacity_1m_v
    WHERE year_num = YEAR(CURRENT_DATE()) AND station_code IS NOT NULL 
      AND (stock_grid_capacity_mw > 0 OR incr_grid_capacity_mw > 0)
    GROUP BY station_code
),
-- 5. 昨日故障容量【季度核心逻辑】：取预测前一天故障值，固定作为未来所有日期故障容量
yesterday_fault AS (
    SELECT
        station_code,
        COALESCE(fault_capacity_mw, 0) AS fixed_fault_capacity
    FROM dws.dws_stnops_powergen_station_daily_1d
    WHERE biz_dt = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
      AND fault_capacity_mw IS NOT NULL
),
-- 6. 启用的停运事项【季度核心逻辑】：去重+过滤启用状态
valid_stop_events AS (
    SELECT DISTINCT
        station_id AS station_code,
        DATE(start_date) AS start_dt,
        DATE(end_date) AS end_dt,
        stop_capacity
    FROM dljy_fill.fill.fill_power_stop_event
    WHERE status = '启用'  AND station_id IS NOT NULL AND stop_capacity IS NOT NULL
),
-- 7. 停运事项生效日期关联【季度核心逻辑】
stop_event_dates AS (
    SELECT
        vse.station_code,
        fd.target_day,
        vse.stop_capacity
    FROM valid_stop_events vse
    INNER JOIN future_dates fd ON fd.target_day BETWEEN vse.start_dt AND vse.end_dt
),
-- 8. 按场站+日期汇总停运容量【季度核心逻辑】
daily_stop AS (
    SELECT
        station_code,
        target_day,
        COALESCE(SUM(stop_capacity), 0) AS stop_capacity
    FROM stop_event_dates
    GROUP BY station_code, target_day
),
-- 9. 损耗率计算【季度核心逻辑】：故障+停运 合并计算损耗率，返回计划停运容量
loss_rate AS (
    SELECT 
        fd.target_day AS cd_target_day,
        vs.station_code,
        ROUND(
            (COALESCE(yf.fixed_fault_capacity, 0) + COALESCE(ds.stop_capacity, 0)) 
            / NULLIF(COALESCE(vs.cumulative_grid_capacity_mw, 1), 0), 
            6
        ) AS loss_rate,
        COALESCE(ds.stop_capacity, 0) AS planned_stop_capacity
    FROM future_dates fd
    CROSS JOIN valid_stations vs
    LEFT JOIN yesterday_fault yf ON vs.station_code = yf.station_code
    LEFT JOIN daily_stop ds ON vs.station_code = ds.station_code AND fd.target_day = ds.target_day
    WHERE fd.forecast_year = vs.capacity_year
    GROUP BY fd.target_day, vs.station_code, yf.fixed_fault_capacity, ds.stop_capacity, vs.cumulative_grid_capacity_mw
),
-- 10. 技改提升小时数【季度核心逻辑】：表名+字段更新，聚合清洗技改数据
power_increase AS (
    SELECT
        station_code,
        increase_year,
        SUM(month_1) AS month_1, SUM(month_2) AS month_2, SUM(month_3) AS month_3,
        SUM(month_4) AS month_4, SUM(month_5) AS month_5, SUM(month_6) AS month_6,
        SUM(month_7) AS month_7, SUM(month_8) AS month_8, SUM(month_9) AS month_9,
        SUM(month_10) AS month_10, SUM(month_11) AS month_11, SUM(month_12) AS month_12,
        MAX(monthly_capacity) AS monthly_capacity,
        increase_year AS capacity_month
    FROM (
        SELECT
            ie.station_id AS station_code,
            CAST(ie.`year` AS SIGNED) AS increase_year,
            COALESCE(ie.month_1,0) month_1,COALESCE(ie.month_2,0) month_2,COALESCE(ie.month_3,0) month_3,
            COALESCE(ie.month_4,0) month_4,COALESCE(ie.month_5,0) month_5,COALESCE(ie.month_6,0) month_6,
            COALESCE(ie.month_7,0) month_7,COALESCE(ie.month_8,0) month_8,COALESCE(ie.month_9,0) month_9,
            COALESCE(ie.month_10,0) month_10,COALESCE(ie.month_11,0) month_11,COALESCE(ie.month_12,0) month_12,
            COALESCE(cm.cumulative_grid_capacity_mw,0) AS monthly_capacity,
            cm.year_num AS year_num
        FROM dljy_fill.fill.fill_power_increase_event ie
        LEFT JOIN ads.ads_stnops_energy_monthly_capacity_1m_v cm
            ON ie.station_id = cm.station_code AND CAST(ie.`year` AS SIGNED) = cm.year_num
        WHERE CAST(ie.`year` AS SIGNED) = YEAR(CURRENT_DATE()) AND ie.station_id IS NOT NULL
    ) t
    GROUP BY station_code, increase_year
),
-- 11. 预测参数汇总【季度核心逻辑】：整合天气系数/权重系数/限发率/技改小时数 所有入参
forecast_params AS (
    SELECT 
        fd.target_day, fd.forecast_year, fd.forecast_month, fd.is_15d_inner,
        DAY(LAST_DAY(fd.target_day)) AS days_of_month,
        vs.station_code,
        vs.stock_capacity, vs.incr_capacity, vs.cumulative_grid_capacity_mw,
        COALESCE(CASE fd.forecast_month
            WHEN 1 THEN eh.month_1 WHEN 2 THEN eh.month_2 WHEN 3 THEN eh.month_3 
            WHEN 4 THEN eh.month_4 WHEN 5 THEN eh.month_5 WHEN 6 THEN eh.month_6
            WHEN 7 THEN eh.month_7 WHEN 8 THEN eh.month_8 WHEN 9 THEN eh.month_9 
            WHEN 10 THEN eh.month_10 WHEN 11 THEN eh.month_11 WHEN 12 THEN eh.month_12
        END / NULLIF(DAY(LAST_DAY(fd.target_day)),0),0) AS hist_avg_hours,
        COALESCE(CASE fd.forecast_month
            WHEN 1 THEN pi.month_1 WHEN 2 THEN pi.month_2 WHEN 3 THEN pi.month_3 
            WHEN 4 THEN pi.month_4 WHEN 5 THEN pi.month_5 WHEN 6 THEN pi.month_6
            WHEN 7 THEN pi.month_7 WHEN 8 THEN pi.month_8 WHEN 9 THEN pi.month_9 
            WHEN 10 THEN pi.month_10 WHEN 11 THEN pi.month_11 WHEN 12 THEN pi.month_12
        END,0) AS raw_month_increase_cap,
        COALESCE(pi.monthly_capacity,0) AS monthly_capacity,
        COALESCE(ROUND((CASE fd.forecast_month
            WHEN 1 THEN pi.month_1 WHEN 2 THEN pi.month_2 WHEN 3 THEN pi.month_3 
            WHEN 4 THEN pi.month_4 WHEN 5 THEN pi.month_5 WHEN 6 THEN pi.month_6
            WHEN 7 THEN pi.month_7 WHEN 8 THEN pi.month_8 WHEN 9 THEN pi.month_9 
            WHEN 10 THEN pi.month_10 WHEN 11 THEN pi.month_11 WHEN 12 THEN pi.month_12
        END) / NULLIF(pi.monthly_capacity,0) / NULLIF(DAY(LAST_DAY(fd.target_day)),0),6),0) AS daily_increase_hours,
        COALESCE(coe.weather_weight_k1,0) k1, COALESCE(coe.hist_weight_k2,0) k2,
        COALESCE(coe.`15d_weather_weight_k3`,0) k3, COALESCE(coe.`15d_hist_weight_k4`,0) k4,
        COALESCE(coe.experience_d1,1) d1, COALESCE(coe.`15d_experience_d2`,1) d2,
        COALESCE(w.forecast_weather,'') weather, COALESCE(w.final_coeff,0) weather_coeff,
        COALESCE(CASE fd.forecast_month
            WHEN 1 THEN limit_tab.month_1 WHEN 2 THEN limit_tab.month_2 WHEN 3 THEN limit_tab.month_3 
            WHEN 4 THEN limit_tab.month_4 WHEN 5 THEN limit_tab.month_5 WHEN 6 THEN limit_tab.month_6
            WHEN 7 THEN limit_tab.month_7 WHEN 8 THEN limit_tab.month_8 WHEN 9 THEN limit_tab.month_9 
            WHEN 10 THEN limit_tab.month_10 WHEN 11 THEN limit_tab.month_11 WHEN 12 THEN limit_tab.month_12
        END,0) forecast_limit_rate
    FROM future_dates fd
    INNER JOIN valid_stations vs ON fd.forecast_year = vs.capacity_year
    LEFT JOIN power_increase pi ON vs.station_code = pi.station_code AND vs.capacity_year = pi.increase_year
    LEFT JOIN dljy_fill.fill.fill_power_monthly_equiv_hours eh ON vs.station_code = eh.station_id
    LEFT JOIN dljy_fill.fill.fill_power_weight_coefficient coe ON vs.station_code = coe.station_id
    LEFT JOIN dljy_fill.fill.fill_power_limit_rate limit_tab ON vs.station_code = limit_tab.station_id AND fd.forecast_year = CAST(limit_tab.`year` AS SIGNED)
    LEFT JOIN ads.ads_stnops_energy_weather_coeff_1d w ON vs.station_code = w.station_code AND fd.target_day = DATE(w.forecast_date)
),
-- ===================== 年度核心：历史实际数据 【必保留】=====================
history_data AS (
    SELECT 
        YEAR(biz_dt) AS `year`,
        MONTH(biz_dt) AS `month`,
        station_code,
        '存量' AS energy_type,
        ROUND(SUM(stock_energy_kwh) / 10000, 6) AS energy_kwh
    FROM dws.dws_stnops_powergen_station_daily_1d
    WHERE YEAR(biz_dt) = YEAR(CURRENT_DATE()) AND biz_dt < CURRENT_DATE() AND stock_energy_kwh IS NOT NULL
    GROUP BY YEAR(biz_dt), MONTH(biz_dt), station_code
    UNION ALL
    SELECT 
        YEAR(biz_dt) AS `year`,
        MONTH(biz_dt) AS `month`,
        station_code,
        '增量' AS energy_type,
        ROUND(SUM(incr_energy_kwh) / 10000, 6) AS energy_kwh
    FROM dws.dws_stnops_powergen_station_daily_1d
    WHERE YEAR(biz_dt) = YEAR(CURRENT_DATE()) AND biz_dt < CURRENT_DATE() AND incr_energy_kwh IS NOT NULL
    GROUP BY YEAR(biz_dt), MONTH(biz_dt), station_code
),
-- ===================== 年度核心：未来预测数据 【复用季度公式】=====================
future_forecast_daily AS (
    SELECT 
        fp.forecast_year AS `year`,
        fp.forecast_month AS `month`,
        fp.station_code,
        fp.stock_capacity,
        fp.incr_capacity,
        ROUND(GREATEST(CASE WHEN fp.is_15d_inner = 1 THEN
            (fp.k1 * fp.weather_coeff + fp.k2 * fp.hist_avg_hours + fp.daily_increase_hours) * fp.d1
        ELSE
            (fp.k3 * fp.weather_coeff + fp.k4 * fp.hist_avg_hours + fp.daily_increase_hours) * fp.d2
        END * fp.stock_capacity *1000 * (1 - fp.forecast_limit_rate - COALESCE(lr.loss_rate,0)),0),4) AS daily_stock_generation,
        ROUND(GREATEST(CASE WHEN fp.is_15d_inner = 1 THEN
            (fp.k1 * fp.weather_coeff + fp.k2 * fp.hist_avg_hours + fp.daily_increase_hours) * fp.d1
        ELSE
            (fp.k3 * fp.weather_coeff + fp.k4 * fp.hist_avg_hours + fp.daily_increase_hours) * fp.d2
        END * fp.incr_capacity *1000 * (1 - fp.forecast_limit_rate - COALESCE(lr.loss_rate,0)),0),4) AS daily_incr_generation
    FROM forecast_params fp
    LEFT JOIN loss_rate lr ON fp.target_day = lr.cd_target_day AND fp.station_code = lr.station_code
),
-- 未来预测按月聚合
future_forecast_month AS (
    SELECT 
        `year`, `month`, station_code,
        '存量' AS energy_type,
        ROUND(SUM(daily_stock_generation) / 10000,6) AS energy_kwh
    FROM future_forecast_daily WHERE stock_capacity > 0
    GROUP BY `year`, `month`, station_code
    UNION ALL
    SELECT 
        `year`, `month`, station_code,
        '增量' AS energy_type,
        ROUND(SUM(daily_incr_generation) / 10000,6) AS energy_kwh
    FROM future_forecast_daily WHERE incr_capacity > 0
    GROUP BY `year`, `month`, station_code
),
-- ===================== 年度核心：历史+未来 合并汇总 【必保留】=====================
combined_data AS (
    SELECT 
        `year`, `month`, station_code, energy_type,
        SUM(energy_kwh) AS total_energy_kwh
    FROM (
        SELECT `year`, `month`, station_code, energy_type, energy_kwh FROM history_data
        UNION ALL
        SELECT `year`, `month`, station_code, energy_type, energy_kwh FROM future_forecast_month
    ) t
    GROUP BY `year`, `month`, station_code, energy_type
),
-- ================ 【核心修改2】万能兼容版 base_dim 存量/增量 全年唯一行 ================
base_dim AS (
    SELECT 
        ym.`year`, 
        ym.`month`, 
        vs.station_code, 
        '存量' AS energy_type,
        vs.stock_capacity AS capacity_mw,
        vs.stock_capacity AS stock_grid_capacity_mw,
        vs.incr_capacity AS incr_grid_capacity_mw,
        vs.cumulative_grid_capacity_mw,
        si.station_name,
        si.station_type,
        si.src_region_name,
        si.src_province_name
    FROM year_month ym 
    INNER JOIN valid_stations vs ON ym.`year`=vs.capacity_year
    INNER JOIN station_info si ON vs.station_code = si.station_code
    WHERE vs.stock_capacity > 0
    
    UNION ALL
    
    SELECT 
        ym.`year`, 
        ym.`month`, 
        vs.station_code, 
        '增量' AS energy_type,
        vs.incr_capacity AS capacity_mw,
        vs.stock_capacity AS stock_grid_capacity_mw,
        vs.incr_capacity AS incr_grid_capacity_mw,
        vs.cumulative_grid_capacity_mw,
        si.station_name,
        si.station_type,
        si.src_region_name,
        si.src_province_name
    FROM year_month ym 
    INNER JOIN valid_stations vs ON ym.`year`=vs.capacity_year
    INNER JOIN station_info si ON vs.station_code = si.station_code
    WHERE vs.incr_capacity > 0
),
-- 基础数据整合
base_data AS (
    SELECT 
        bd.`year`, 
        bd.`month`, 
        bd.station_code, 
        bd.energy_type,
        COALESCE(cd.total_energy_kwh, 0) AS energy_kwh,
        bd.capacity_mw,
        bd.stock_grid_capacity_mw,
        bd.incr_grid_capacity_mw,
        bd.cumulative_grid_capacity_mw,
        bd.station_name,
        bd.station_type,
        bd.src_region_name,
        bd.src_province_name
    FROM base_dim bd
    LEFT JOIN combined_data cd 
        ON bd.`year`=cd.`year` AND bd.`month`=cd.`month` AND bd.station_code=cd.station_code AND bd.energy_type=cd.energy_type
),
-- ===================== 最终年度结果：12个月透视展示 =====================
final_result AS (
    SELECT 
        CONCAT('V', YEAR(CURRENT_DATE()), LPAD(MONTH(CURRENT_DATE()), 2, '0'), DATE_FORMAT(CURRENT_TIMESTAMP(), '%d')) AS version,
        `year`,
        station_code,
        energy_type,
        COALESCE(station_name, '') AS station_name,
        COALESCE(station_type, '') AS station_type,
        COALESCE(src_region_name, '') AS src_region_name,
        COALESCE(src_province_name, '') AS src_province_name,
        COALESCE(capacity_mw, 0) AS capacity,
        COALESCE(cumulative_grid_capacity_mw, 0) AS cumulative_grid_capacity,
        MAX(CASE WHEN `month` = 1 THEN energy_kwh ELSE 0 END) AS month_01,
        MAX(CASE WHEN `month` = 2 THEN energy_kwh ELSE 0 END) AS month_02,
        MAX(CASE WHEN `month` = 3 THEN energy_kwh ELSE 0 END) AS month_03,
        MAX(CASE WHEN `month` = 4 THEN energy_kwh ELSE 0 END) AS month_04,
        MAX(CASE WHEN `month` = 5 THEN energy_kwh ELSE 0 END) AS month_05,
        MAX(CASE WHEN `month` = 6 THEN energy_kwh ELSE 0 END) AS month_06,
        MAX(CASE WHEN `month` = 7 THEN energy_kwh ELSE 0 END) AS month_07,
        MAX(CASE WHEN `month` = 8 THEN energy_kwh ELSE 0 END) AS month_08,
        MAX(CASE WHEN `month` = 9 THEN energy_kwh ELSE 0 END) AS month_09,
        MAX(CASE WHEN `month` = 10 THEN energy_kwh ELSE 0 END) AS month_10,
        MAX(CASE WHEN `month` = 11 THEN energy_kwh ELSE 0 END) AS month_11,
        MAX(CASE WHEN `month` = 12 THEN energy_kwh ELSE 0 END) AS month_12,
        CURRENT_TIMESTAMP AS create_time
    FROM base_data
    GROUP BY 
        `year`, 
        station_code, 
        energy_type,
        station_name, 
        station_type, 
        src_region_name,
        src_province_name, 
        capacity_mw, 
        cumulative_grid_capacity_mw
    HAVING 
        (month_01 + month_02 + month_03 + month_04 + month_05 + month_06 + 
         month_07 + month_08 + month_09 + month_10 + month_11 + month_12) > 0
)
-- 最终查询
SELECT * FROM final_result
ORDER BY `year`, energy_type DESC, station_code
LIMIT 1000;